In [3]:
import pandas as pd
import shap
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score

In [4]:
# Load datasets
train_data = pd.read_csv(f'../dataset_classification/TrainDataset.csv')
test_data = pd.read_csv(f'../dataset_classification/TestDataset.csv')

#Clean TR from features
x_train = train_data.drop(columns = ["label", "position"])
y_train = train_data["label"]

#Split TR in TR and VL with HoldOut technique
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

#Clean TS from features
x_test = test_data.drop(columns = ["label", "position"])
y_test = test_data["label"]

In [5]:
# Define model
clf = LogisticRegression(verbose=1, max_iter=500, random_state=42)

# Train model
print("Starting training for StackingClassifier with linear models...")
clf.fit(x_train, y_train)
print("Finished training.")

# Evaluate model
print("")
print("Final evaluation")
print("")

train_accuracy = accuracy_score(y_train, clf.predict(x_train))
val_accuracy = accuracy_score(y_val, clf.predict(x_val))
test_accuracy = accuracy_score(y_test, clf.predict(x_test))

print(f"Train accuracy: {train_accuracy}")
print(f"Validation accuracy: {val_accuracy}")
print(f"Test accuracy: {test_accuracy}")
print("")

# Calculate log-loss
train_y_proba = clf.predict_proba(x_train)
val_y_proba = clf.predict_proba(x_val)
test_y_proba = clf.predict_proba(x_test)

train_loss = log_loss(y_train, train_y_proba)
val_loss = log_loss(y_val, val_y_proba)
test_loss = log_loss(y_test, test_y_proba)

print(f"Train Log-Loss: {train_loss}")
print(f"Val Log-Loss: {val_loss}")
print(f"Test Log-Loss: {test_loss}")
print("")

# Calculate AUC
train_auc = roc_auc_score(y_train, train_y_proba[:, 1])
val_auc = roc_auc_score(y_val, val_y_proba[:, 1])
test_auc = roc_auc_score(y_test, test_y_proba[:, 1])

print(f"Train AUC: {train_auc}")
print(f"Val AUC: {val_auc}")
print(f"Test AUC: {test_auc}")
print("")

# SHAP Integration -------------------------------------------------------------------
print("Starting SHAP analysis...")

# Apply SHAP to model
explainer = shap.Explainer(clf, x_train)
shap_values = explainer(x_train)

# Visualize SHAP analysis to model
shap.summary_plot(shap_values, x_train)

print("Finished SHAP analysis.")

Starting training for StackingClassifier with linear models...


ValueError: could not convert string to float: 'tour-de-france/2022/stage-3'